In [1]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchmetrics
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import lightning as L

/opt/miniconda3/envs/real-estate-dinov2-test/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/miniconda3/envs/real-estate-dinov2-test/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /opt/miniconda3/envs/real-estate-dinov2-test/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <761A4B43-4CD1-322C-BB16-CEE783FE0A7C> /opt/miniconda3/envs/real-estate-dinov2-test/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
df = pd.read_pickle('./data/flats_train.pkl')
df

,Numer oferty,Liczba pokoi,Piętro,Liczba pięter,Powierzchnia,Czynsz administracyjny,Stan nieruchomości,Umeblowanie,Rodzaj budynku,Rok budowy,...,Typ_mieszkania_Jednopoziomowe,Balkon,Alarm,Monitoring,Ochrona,Recepcja,Teren ogrodzony,Liczba oddzielnych toalet,Cena,Cena za m2
0,23159/3685/OMS,1,1,7,26.80,350.0,8,3,5,2010.0,...,False,0,0,1,1,1,1,0,420000.0,15672.0
1,31211/3685/OMS,2,4,4,36.98,389.0,0,3,8,2018.0,...,False,1,0,1,0,0,1,0,490000.0,13250.0
2,26529/3685/OMS,2,3,4,48.47,1064.0,8,3,2,2006.0,...,False,1,0,1,0,1,1,0,900000.0,18568.0
4,24977/3685/OMS,4,2,4,97.34,800.0,9,3,2,2006.0,...,True,1,0,1,0,0,0,0,1270000.0,13047.0
6,27440/3685/OMS,2,1,4,39.34,720.0,9,3,8,2021.0,...,False,1,0,1,1,1,1,0,599000.0,15226.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,24780/3685/OMS,3,3,4,45.30,530.0,6,0,8,1970.0,...,True,1,0,1,0,0,0,0,370000.0,8168.0
65,28798/3685/OMS,4,6,7,103.38,1342.0,0,0,0,2001.0,...,False,1,0,0,0,0,0,0,980000.0,9480.0
66,28566/3685/OMS,3,3,4,67.70,800.0,0,3,6,1995.0,...,False,1,0,1,0,0,1,0,830000.0,12260.0
67,23471/3685/OMS,2,2,4,46.82,500.0,7,2,8,2023.0,...,True,1,0,0,0,0,1,0,550000.0,11747.0


In [27]:
class CustomDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        offer_id = row['Numer oferty']

        embedded_filename = '_'.join(offer_id.split("/"))
        # print(f"./images_embeded/{embedded_filename}.pkl")
        with open(f"./images_embeded/{embedded_filename}.pkl", 'rb') as f:
            embedding = pickle.load(f)['outputs']
        embedding = torch.tensor(embedding, dtype=torch.float32)

        features = row.iloc[1:-2]
        features = torch.tensor(features, dtype=torch.float32)

        price = row['Cena']
        price = torch.tensor(price, dtype=torch.float32)

        price_per_meter = row['Cena za m2']
        price_per_meter = torch.tensor(price_per_meter, dtype=torch.float32)
        
        return features, embedding, price, price_per_meter
    
dataset = CustomDataset(df)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

for features, embedding, price, price_per_meter in dataloader:
    print(features.shape)
    print(embedding.shape)
    print(price)
    print(price_per_meter)
    break

torch.Size([1, 73])
torch.Size([1, 23, 384])
tensor([830000.])
tensor([12260.])


/var/folders/_s/r6q44yzn7gd7hjnfjjlpb4hm0000gn/T/ipykernel_59740/2770139682.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  features = torch.tensor(features, dtype=torch.float32)
